```{.dockerfile include="../_version.md"}
```

In [1]:
#| output: false
#| echo: false
docker system prune --all --force

Deleted Images:


untagged: ubuntu:jammy


untagged: ubuntu@sha256:f9d633ff6640178c2d0525017174a688e2c1aef28f0a0130b26bd5554491f0da


deleted: sha256:a50ab9f167975489853cbffd2be3bcadab3a9da27faf390ac48603c60d5c59e7


deleted: sha256:98d1994bf6c1db82b81822e3b2f8fdb6933c44fcd9487787c41ee907368022e2


Total reclaimed space: 69.24MB


Le format d'une image est défini par un [standard ouvert]( https://github.com/opencontainers/image-spec/blob/main/spec.md). En plus de différentes métadonnées comme la command par défaut, une image est composée d'un ensemble de couches qui indiquent les changements effectués à partir de la couche précédente. Cela permet de partager efficacement des bases communes entre images mais demande quelques précautions.

## Manuellement

La commande `history` montre les couches qui composent une image.

In [2]:
docker pull ubuntu:jammy
docker history ubuntu:jammy

jammy: Pulling from library/ubuntu


Digest: sha256:f9d633ff6640178c2d0525017174a688e2c1aef28f0a0130b26bd5554491f0da


Status: Downloaded newer image for ubuntu:jammy


docker.io/library/ubuntu:jammy


IMAGE          CREATED       CREATED BY                                      SIZE      COMMENT


...

<missing>      3 weeks ago   /bin/sh -c #(nop) ADD file:8d91b8bd386e0cc34…   69.2MB    


<missing>      3 weeks ago   /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        


<missing>      3 weeks ago   /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        


<missing>      3 weeks ago   /bin/sh -c #(nop)  ARG LAUNCHPAD_BUILD_ARCH     0B        


<missing>      3 weeks ago   /bin/sh -c #(nop)  ARG RELEASE                  0B        


Pour créer une image, il est possible d'exécuter un conteneur, de faire de faire des modifications manuelles puis de les valider. Cela provoque la création d'une nouvelle couche dans le système de fichier et donc d'une nouvelle image. Cette approche est déconseillée car difficile à maintenir (par exemple lors d'un simple changement de version d'un logiciel). 

On peut ainsi fabriquer un conteneur simple pour fournir la commande `git`.

En exécutant manuellement les commandes dans un conteneur

In [3]:
docker run --name my-ubuntu-container --interactive ubuntu:jammy bash -  <<EOF
apt-get update &> /dev/null
apt-get install --yes --no-install-recommends &> /dev/null \
    git 
apt-get clean autoclean &> /dev/null
apt-get autoremove --yes &> /dev/null
rm -rf /var/lib/apt/lists/*
EOF

puis en validant ces modifications sous la forme d'une nouvelle couche d'une image

In [4]:
docker commit my-ubuntu-container mygit:latest

sha256:2a6d79d6af7ae0215f2537ac5b581738d9fbce15d4c8f762a00b648b1ae3ae9a


le conteneur modifié peut alors être supprimé

In [5]:
docker rm my-ubuntu-container

my-ubuntu-container


et la nouvelle image utilisée pour créer autant de conteneur que nécessaire

In [6]:
docker run --rm mygit git --version

git version 2.34.1


Nous pouvons voir que l'ID de l'avant-dernière couche est celui de la dernière de l'image d'ubuntu et qu'une nouvelle couche qui contient les modifications faite par les commandes précédentes a été ajoutée.

In [7]:
docker history mygit

IMAGE          CREATED        CREATED BY                                      SIZE      COMMENT


2a6d79d6af7a   1 second ago   bash -                                          64.2MB    


a50ab9f16797   3 weeks ago    /bin/sh -c #(nop)  CMD ["/bin/bash"]            0B        


<missing>      3 weeks ago    /bin/sh -c #(nop) ADD file:8d91b8bd386e0cc34…   69.2MB    


<missing>      3 weeks ago    /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        


<missing>      3 weeks ago    /bin/sh -c #(nop)  LABEL org.opencontainers.…   0B        


<missing>      3 weeks ago    /bin/sh -c #(nop)  ARG LAUNCHPAD_BUILD_ARCH     0B        


<missing>      3 weeks ago    /bin/sh -c #(nop)  ARG RELEASE                  0B        


In [8]:
docker image rm mygit

Untagged: mygit:latest


Deleted: sha256:2a6d79d6af7ae0215f2537ac5b581738d9fbce15d4c8f762a00b648b1ae3ae9a


Deleted: sha256:2659c1d95680d7d8d7b9ea7faed18db00b3c281f288c1d06633dd31936bd12f3


## Dockerfile

L'approche recommandée est de créer un fichier `Dockerfile` (cf. https://docs.docker.com/engine/reference/builder/) pour automatiser la création d'une image. Ce fichier peut être édité lors des mises à jour et versionné avec git pour gérer un historique de l'infrastructure ainsi que des versions parallèles.

### Structure de base

Le fichier `Dockerfile` suivant illustre les concepts de base. Il s'appuie sur l'image `python:3.11-slim-bullseye` (`FROM`), ajoute les fichiers de l'application depuis le contexte (`COPY`), exécute les commandes nécessaire à la construction de l'image (`RUN`) et défini la commande par défaut de l'image (`CMD`).

```{.dockerfile include="sample-python/helloworld/Dockerfile" code-line-numbers="true"}
```

La commande suivante construit une image tagguée à partir du Dockerfile contenu dans le répertoire courant (indiqué par le `.` à la fin).
C'est ce dernier paramètre qui est le contexte. Les options `--build-arg` permettre de fixer les `ARG` du Dockerfile.

In [9]:
( cd sample-python/helloworld && \
 docker image build \
     --quiet \
     --build-arg BUILD_DATE=$(date -u +'%Y-%m-%dT%H:%M:%SZ')\
     --tag brunoe/helloworld_python:0.1 \
     . 
)

cd: no such file or directory: sample-python/helloworld


d'autres tags peuvent être appliqués pour facilier l'usage de l'image, généralement des versions plus génériques (1.2.3, 1.2 et 1) et/ou des informations sur l'image de base avec des variantes (1.2.3-ubuntu, 1.2.3-debian, ...) et/ou des variantes de constructions de l'image elle-même et toujours `latest`.

In [10]:
docker tag brunoe/helloworld_python:0.1 brunoe/helloworld_python:0
docker tag brunoe/helloworld_python:0.1 brunoe/helloworld_python:latest

Error response from daemon: No such image: brunoe/helloworld_python:0.1


Error response from daemon: No such image: brunoe/helloworld_python:0.1


Cette image peut alors être utilisée localement.

In [11]:
docker run --rm brunoe/helloworld_python:0.1

Unable to find image 'brunoe/helloworld_python:0.1' locally


docker: Error response from daemon: pull access denied for brunoe/helloworld_python, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


In [12]:
docker run --env NAME="Pierre" --rm brunoe/helloworld_python:0.1

Unable to find image 'brunoe/helloworld_python:0.1' locally


docker: Error response from daemon: pull access denied for brunoe/helloworld_python, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


Il est intéressant de connaitre la taille d'une image et le détail des couches qui la compose. 

In [13]:
docker image ls|grep brunoe/helloworld_python|tr -s ' '

In [14]:
docker history brunoe/helloworld_python:0.1

Error response from daemon: No such image: brunoe/helloworld_python:0.1


### Partage de l'image

Une image peut ensuite être partagée sur un registry (dockerhub ou un autre qui peut être public ou privé). Pour cela le docker client doit s'authentifier (`docker login`) puis pousser les différents tags de l'image (`docker push <image_name>`). Attention, le nom de l'image doit généralement préciser le compte sur le repository.

```bash
docker push brunoe/helloworld_python:0.1
docker push brunoe/helloworld_python:latest
```

## Entrypoint

La directive `ENTRYPOINT` prend en paramètre une commande permet que l'image docker se comporte comme cette commande en acceptant directement les paramètres lors d'un `run`. La commande exécutée est la concaténation de `ENTRYPOINT` et des paramètre de `run` ou de `CMD`. `CMD` correspond donc aux options par défaut que l'on souhaite si aucun paramètre n'est donné. 

L'exemple suivant montre comment construire une commande qui crée une base de donnée sqlite dans repertoire courant.

```{.dockerfile include="sample-sqlite/Dockerfile" code-line-numbers="true"}
```

In [15]:
docker build --quiet --tag sqlite sample-sqlite/

ERROR: unable to prepare context: path "sample-sqlite/" not found


Le run affiche par défaut la version au lieu de l'aide.

In [16]:
docker run --rm sqlite

Unable to find image 'sqlite:latest' locally


docker: Error response from daemon: pull access denied for sqlite, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


et il n'est plus nécessaire de redonner la commande, le nom de l'image suffit. En utilisant un alias, il est donc possible de donner l'illusion qu'il s'agit d'un programme.

In [17]:
# sqlite() {docker run -v ${PWD}:/workdir --rm sqlite}
sqlite() {docker run -v /tmp:/workdir --rm sqlite}
    
sqlite "DROP TABLE IF EXISTS data;"
sqlite "CREATE TABLE IF NOT EXISTS data (key TEXT PRIMARY KEY,value TEXT NOT NULL);"
sqlite "INSERT INTO data values('A','10');INSERT INTO data values('B','20');"
sqlite "SELECT * FROM data;"

Unable to find image 'sqlite:latest' locally


docker: Error response from daemon: pull access denied for sqlite, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


Unable to find image 'sqlite:latest' locally


docker: Error response from daemon: pull access denied for sqlite, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


...

docker: Error response from daemon: pull access denied for sqlite, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


Unable to find image 'sqlite:latest' locally


docker: Error response from daemon: pull access denied for sqlite, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


## Multistage

Un Dockerfile dit `multistage` utilise plusieurs `FROM`. Chacun définit une image mais seule la dernière sera le résultat final. Les images précédentes peuvent être utilisées pour produire des fichiers qui seront copiés dans une étape (stage) suivant. 

Pour cela, chaque `FROM` peut ête nommé avec `AS` pour qu'une étape suivante puisse accéder à son système de fichier avec `COPY --from=<stage_name> <source> <destination>`. 

L'utilisation la plus classique est d'avoir deux stages : (1) compilation et (2) exécution. La première étape contient l'environnement de développement (gcc pour le C, JDK+Maven pour Java, ...) et les sources,  elle produit l'exécutable (un binaire pour le C, du bytecode ou un jar pour Java, ...). La seconde étape ne contient que l'environnement d'exécution (éventuelles librairies dynamiques pour le C, JRE pour Java, ...) et l'exécutable.

```{.dockerfile include="sample-c/helloworld/Dockerfile" code-line-numbers="true"}
```

In [18]:
( cd sample-c/helloworld/ && \
 docker image build \
     --quiet \
     --tag brunoe/helloworld_c:0.1 \
     . )

cd: no such file or directory: sample-c/helloworld/


In [19]:
docker run brunoe/helloworld_c:0.1 Pierre

Unable to find image 'brunoe/helloworld_c:0.1' locally


docker: Error response from daemon: pull access denied for brunoe/helloworld_c, repository does not exist or may require 'docker login': denied: requested access to the resource is denied.


See 'docker run --help'.


### Volumes et réseaux

EXPOSE et VOLUME

L'instruction `VOLUME <chemin dans le conteneur>` définit ce chemin comme un point de montage pour un volume. Cela signifie que les données stockées dans ce répertoire seront persistantes même si le conteneur est supprimé.

L'instruction `EXPOSE <PORT>` expose le port du conteneur aux autres conteneurs.
